In [47]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_od
from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob



run_id='r1dd37mcxv5pf4'
work_dir='data/transcriptic'

if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [21]:
Connection.from_file("~/.transcriptic")
r = tx_od.get_tx_run(run_id)
data = r.data

Attempting to fetch 7 datasets...


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/transcriptic/jupyter/objects.py:437: UserWarning: Missing key 'container' when initializing dataset
  warnings.warn("Missing key {} when initializing dataset".format(e))


In [22]:
data

,Name,DataType,Operation,AnalysisTool,Datasets
0,FLUOR-0.0 hour,platereader,fluorescence,None,<transcriptic.jupyter.objects.Dataset object a...
1,OD-0.0 hour,platereader,absorbance,None,<transcriptic.jupyter.objects.Dataset object a...
2,FLUOR-1.0 hour,platereader,fluorescence,None,<transcriptic.jupyter.objects.Dataset object a...
3,OD-1.0 hour,platereader,absorbance,None,<transcriptic.jupyter.objects.Dataset object a...
4,Flow (Sytox) Plate 1_1.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object a...
5,OD-post-inoculation-read,platereader,absorbance,None,<transcriptic.jupyter.objects.Dataset object a...
6,FLUOR-post-inoculation-read,platereader,fluorescence,None,<transcriptic.jupyter.objects.Dataset object a...


In [49]:
dataset = data.loc[data['Operation'] == 'flow_analyze']['Datasets'].values[0]
key = dataset.attributes["attachments"][0]['key']
tx_config = json.load(open(os.path.join(expanduser("~"), ".transcriptic")))
headers = {'X-User-Email': tx_config['email'], 'X-User-Token': tx_config['token']}
response = requests.get("https://secure.transcriptic.com/upload/url_for?key="+ key, headers=headers)
print("Requesting Data from: " + "https://secure.transcriptic.com/upload/url_for?key="+ key)
zip_path = os.path.join(work_dir, 'fcs.zip')
unzip_path = os.path.join(work_dir, 'fcs')
unzip_tmp_path = os.path.join(work_dir, 'fcs_tmp')
with open(zip_path, 'wb') as f:
    print("Writing data to: " + zip_path)
    print("Response: " + str(response))
    f.write(response.content)
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall(unzip_tmp_path)
zip_ref.close()

# Search for the fcs files
matches = []
for root, dirnames, filenames in os.walk(unzip_tmp_path):
    for filename in fnmatch.filter(filenames, '*.fcs'):
        matches.append((os.path.join(root, filename), filename))

if not os.path.exists(unzip_path):
    os.makedirs(unzip_path)

# Move all fcs files to a directory with a known name
for full_path, filename in matches:
    shutil.move(full_path, os.path.join(unzip_path, filename))

shutil.rmtree(unzip_tmp_path)
os.remove(zip_path)

Requesting Data from: https://secure.transcriptic.com/upload/url_for?key=uploads/9fcf842f-3e37-4b27-8906-d85a18d2326e/20190725_Cell_Death.zip
Writing data to: data/transcriptic/fcs.zip
Response: <Response [200]>
